# 1. Let's build the transformer's encoder

## 1.0. Init

- Initialize the notebook environment.
- Hyperparameters definition

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
# Hyperparameters Here
BATCH_SIZE = 128
T = 80                                  # sentence length
D_K = 16
D_V = 16
D_MODEL = 128
H = 8
VOCAB_SIZE = 10
N_MHA_BLOCKS_ENCODER = 6
N_CLASSES = 2                          # classes of classifier

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"

RUN_UNIT_TESTS = True
TRAIN_MODEL = False

## 1.1. Attention

In [3]:
# Test softmax x axis:
matrix = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])

softmaxed_matrix = F.softmax(matrix, dim=1)

print(softmaxed_matrix)

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652]])


In [4]:
def repeat(x: torch.Tensor, n: int):
    # make shape (n, 1, 1, ...) --> quantity of 1's must be len(x.shape)
    # for example, if shape of x is (3, 4, 8), shapee must be (n, 1, 1, 1)
    tuple_ones = tuple(
        (torch.tensor(x.shape) / torch.tensor(x.shape)).numpy().astype(int)
    )
    return x.unsqueeze(0).repeat((n, *tuple_ones))


def batched_matmul(tensor_3d, tensor_2d):
    W_repeated = repeat(tensor_2d, n=tensor_3d.shape[0])
    
    return torch.bmm(tensor_3d, W_repeated)

In [5]:
class Attention(nn.Module):
    """Convention from: https://www.udemy.com/course/data-science-transformers-nlp/learn/lecture/32255056#overview
    In our convention, K, Q and V are learneable, different from the "Attention is all you need" paper.
    """

    def __init__(self, d_K, d_V, d_model: int, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        # Define trainable parameters with requires_grad=True
        # torch 2d tensor initialized normally
        self.W_K = nn.Parameter(torch.normal(mean=0, std=0.01, size=(d_model, d_K)))
        self.W_Q = nn.Parameter(torch.normal(mean=0, std=0.01, size=(d_model, d_K)))
        self.W_V = nn.Parameter(torch.normal(mean=0, std=0.01, size=(d_model, d_V)))
        self.mask = None

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass of the Attention layer.

        Args:
            x (torch.Tensor): Input tensor of shape (batch, T, d_model)

        Returns:
            torch.Tensor: Output tensor of shape (batch, T, d_V)
            torch.Tensor: Attention weights of shape (batch, T, T)
        """
        # Shapes:
        # x is a 3d tensor (batch, T, d_model)
        # W_K (d_model, d_K)

        # (batch, T, d_model) x (d_model, d_K) -> (batch, T, d_K)
        K = batched_matmul(x, self.W_K)
        Q = batched_matmul(x, self.W_Q)
        V = batched_matmul(x, self.W_V)

        # (batch, T, d_K) x (batch, d_k, T) -> (batch, T, T)
        result = torch.bmm(Q, K.transpose(1, 2)) / (K.shape[-1] ** 0.5)
        if self.mask:
            result = torch.bmm(result, self.mask)
        attention_weights = F.softmax(result, dim=-1)
        # (batch, T, T) x (batch, T, d_V) -> (batch, T, d_V)
        result = torch.bmm(attention_weights, V)
        return result, attention_weights

In [6]:
att = Attention(d_K=D_K, d_V=D_V, d_model=D_MODEL)
x = torch.normal(mean=0, std=0.01, size=(BATCH_SIZE, T, D_MODEL))

att_result, att_weights = att.forward(x)
assert att_result.shape == (BATCH_SIZE, T, D_V)
print(att.W_K.shape)
print(att.W_Q.shape)
print(att.W_V.shape)
print(att_result.shape)

torch.Size([128, 16])
torch.Size([128, 16])
torch.Size([128, 16])
torch.Size([128, 80, 16])


## 1.2. Multi-Head Attention

In [7]:
class MultiHeadAttention(nn.Module):
    def __init__(self, h, d_model, d_K, d_V):
        super().__init__()
        self.h = h
        self.attentions = nn.ModuleList(
            [Attention(d_K=d_K, d_V=d_V, d_model=d_model) for _ in range(h)]
        )
        self.W_O = nn.Parameter(torch.randn(h * d_V, d_model))

    def forward(self, x: torch.Tensor):
        """Forward pass of the MultiHeadAttention layer.

        Args:
            x (torch.Tensor): Input tensor of shape (batch, T, d_model)

        Returns:
            torch.Tensor: Output tensor of shape (batch, T, d_model)
            torch.Tensor: Attention weights of shape (batch, h, T, T)
        """
        attention_results = [attention(x)[0] for attention in self.attentions]
        attention_weights = torch.stack([attention(x)[1] for attention in self.attentions])
        concatenated = torch.cat(attention_results, dim=-1)
        return batched_matmul(concatenated, self.W_O), attention_weights

In [8]:
mha = MultiHeadAttention(h=H, d_K=D_K, d_V=D_V, d_model=D_MODEL)
out, att_weights = mha(x)
assert out.shape == (BATCH_SIZE, T, D_MODEL)

## 1.3. The transformer block

In [9]:
class TransformerBlock(nn.Module):
    def __init__(
        self, T=T, d_K=D_K, d_V=D_V, d_model=D_MODEL, h=H, dropout=0.1, *args, **kwargs
    ) -> None:
        super().__init__(*args, **kwargs)
        self.mha = MultiHeadAttention(h, d_K=d_K, d_V=d_V, d_model=d_model)
        self.layer_norm = nn.LayerNorm(d_model)

        self.ann = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(d_model, d_model),
            nn.Dropout(dropout),
        )

    def forward(self, x: torch.Tensor):
        x = self.layer_norm(x + self.mha(x)[0])
        x = self.layer_norm(x + self.ann(x)[0])
        return x


transformerBlock = TransformerBlock()
transformerBlock(x).shape

torch.Size([128, 80, 128])

## 1.4. The positional encoding

In [10]:
def PositionalEncoding(T: int, d_model) -> torch.Tensor:
    encodings = torch.zeros(size=(T, d_model), requires_grad=False)
    counter = 0
    for pos in range(T):
        for i in range((d_model // 2) + 1):
            if 2 * i < d_model:
                counter += 1
                encodings[pos, 2 * i] = torch.sin(
                    pos / torch.tensor(10000).pow(2 * i / d_model)
                )
            if 2 * i + 1 < d_model:
                counter += 1
                encodings[pos, 2 * i + 1] = torch.cos(
                    pos / torch.tensor(10000).pow(2 * i / d_model)
                )
    assert counter == T * d_model
    return encodings


PositionalEncoding(T, D_MODEL).shape

torch.Size([80, 128])

In [11]:
torch.range(0, 10).reshape(-1, 1)

/tmp/ipykernel_80174/2721896283.py:1: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  torch.range(0, 10).reshape(-1, 1)


tensor([[ 0.],
        [ 1.],
        [ 2.],
        [ 3.],
        [ 4.],
        [ 5.],
        [ 6.],
        [ 7.],
        [ 8.],
        [ 9.],
        [10.]])

## 1.5. The embedding layer

In [12]:

# TODO: make this work in batches
class Embedding(nn.Module):
    def __init__(self, vocab_size, d_model, padding_idx=0):
        super(Embedding, self).__init__()
        self.embedding = nn.Parameter(
            torch.normal(
                mean=0.0, std=0.1, size=(vocab_size, d_model), requires_grad=True
            )
        )

    def forward(self, x):
        """forward pass Embedding layer

        Args:
            x (torch.Tensor): input Tensor of shape (batch_size, T)

        Returns:
            torch.Tensor: output Tensor of shape (batch_size, T, d_model)
        """
        return self.embedding[x.long()]


emb = Embedding(vocab_size=3, d_model=2)
# emb = nn.Embedding(num_embeddings=3, embedding_dim=2)
for name, param in emb.named_parameters():
    print(name, param.data)

input_emb = torch.FloatTensor([[2, 1], [1, 0], [0, 2]])

print(input_emb.shape)

out_emb = emb(input_emb.long())
print(out_emb.shape)
print(out_emb)

embedding tensor([[ 0.0329, -0.2038],
        [-0.0100, -0.1585],
        [-0.1542, -0.0964]])
torch.Size([3, 2])
torch.Size([3, 2, 2])
tensor([[[-0.1542, -0.0964],
         [-0.0100, -0.1585]],

        [[-0.0100, -0.1585],
         [ 0.0329, -0.2038]],

        [[ 0.0329, -0.2038],
         [-0.1542, -0.0964]]], grad_fn=<IndexBackward0>)


## 1.6. The Classification Encoder

In [13]:
class ClassifierEncoder(nn.Module):
    def __init__(
        self,
        T=T,
        d_K=D_K,
        d_V=D_V,
        d_model=D_MODEL,
        h=H,
        vocab_size=VOCAB_SIZE,
        n_classes=N_CLASSES,
        dropout=0.1,
        *args,
        **kwargs
    ):
        super(ClassifierEncoder, self).__init__()
        self.T = T
        self.d_K = d_K

        self.embedding = Embedding(vocab_size, d_model)
        # self.embedding = nn.Embedding(vocab_size, d_model)
        self.position_encoding: torch.Tensor = PositionalEncoding(T, d_model).to(device)

        self.transformer_blocks = nn.ModuleList()
        for _ in range(N_MHA_BLOCKS_ENCODER):
            self.transformer_blocks.append(
                TransformerBlock(T=T, d_K=d_K, d_V=d_V, d_model=d_model, h=h)
            )

        self.prediction_head = nn.Linear(d_model, n_classes)

    def forward(self, x: torch.Tensor):
        positionalEncoding = self.position_encoding.repeat(x.shape[0], 1, 1)
        x = positionalEncoding + self.embedding(x.int()).float()

        for block in self.transformer_blocks:
            x = block(x)

        x = self.prediction_head(x)

        x = x[:, -1, :]

        x = F.softmax(x, dim=-1)
        # Select the last value along the T dimension
        # Because classification is only obtained at the end of the sequence

        return x

# 2. Let's try a classification problem using the CLassification Encoder

* See the file: Fine-Tuning (Intermediate)/Fine-Tunning Sentiment Custom Dataset + Labels.ipynb

## First, lets make a dataset with cross validation

In [14]:
import numpy as np
import pandas as pd

In [15]:
df_ = pd.read_csv("../Fine-Tuning (Intermediate)/AirlineTweets.csv")
df_.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [16]:
df = df_[['airline_sentiment', 'text']].copy()
df.head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [17]:
target_map = {
    'positive': 1,
    'negative': 0,
    'neutral': 2,
}
df['target'] = df['airline_sentiment'].map(target_map)
df.head()

,airline_sentiment,text,target
0,neutral,@VirginAmerica What @dhepburn said.,2
1,positive,@VirginAmerica plus you've added commercials t...,1
2,neutral,@VirginAmerica I didn't today... Must mean I n...,2
3,negative,@VirginAmerica it's really aggressive to blast...,0
4,negative,@VirginAmerica and it's a really big bad thing...,0


In [18]:
df_filtered = df[df['target'] != 2]
df_filtered.head()

,airline_sentiment,text,target
1,positive,@VirginAmerica plus you've added commercials t...,1
3,negative,@VirginAmerica it's really aggressive to blast...,0
4,negative,@VirginAmerica and it's a really big bad thing...,0
5,negative,@VirginAmerica seriously would pay $30 a fligh...,0
6,positive,"@VirginAmerica yes, nearly every time I fly VX...",1


In [19]:
df2 = df_filtered[['text', 'target']]
# Not documented info: targets must have the column name label
# sentence may have other names, but not label
df2.columns = ['sentence', 'label']
df2.to_csv("data.csv", index=False)
!head data.csv
# df2.head()

sentence,label
@VirginAmerica plus you've added commercials to the experience... tacky.,1
"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' faces &amp; they have little recourse",0
@VirginAmerica and it's a really big bad thing about it,0
"@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing.
it's really the only bad thing about flying VA",0
"@VirginAmerica yes, nearly every time I fly VX this “ear worm” won’t go away :)",1
"@virginamerica Well, I didn't…but NOW I DO! :-D",1
"@VirginAmerica it was amazing, and arrived an hour early. You're too good to me.",1
@VirginAmerica I &lt;3 pretty graphics. so much better than minimal iconography. :D,1


In [20]:
from datasets import load_dataset
raw_dataset = load_dataset('csv', data_files="data.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [21]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 11541
    })
})

In [22]:
from datasets import Dataset, DatasetDict

def splitTrainTestValidation(dataset: Dataset, valid_size=.1, test_size=.1):
    len_valid = int(len(dataset) * valid_size)
    len_test = int(len(dataset) * test_size)
    
    splited: DatasetDict = dataset.train_test_split(len_valid + len_test, shuffle=False, seed=42)
    splited['validation'] = splited['test']
    del splited['test']
    
    splited_2 = splited['validation'].train_test_split(len_test, shuffle=True, seed=42)
    splited['validation'] = splited_2['train']
    splited['test'] = splited_2['test']
    
    return splited

In [23]:
# split = raw_dataset['train'].train_test_split(test_size=.3, seed=42)
split = splitTrainTestValidation(raw_dataset['train'], valid_size=.1, test_size=.1)

In [24]:
split

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 9233
    })
    validation: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1154
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 1154
    })
})

In [25]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-cased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_fn(batch):
    return tokenizer(batch["sentence"], truncation=True, padding='max_length', max_length=T)


# tokenizer("This is an example", truncation=True, padding=True)

In [26]:

tokenized_datasets = split.map(tokenize_fn, batched=False)
tokenized_datasets

Map:   0%|          | 0/9233 [00:00<?, ? examples/s]

Map:   0%|          | 0/1154 [00:00<?, ? examples/s]

Map:   0%|          | 0/1154 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 9233
    })
    validation: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1154
    })
    test: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1154
    })
})

In [27]:
# Little notation here:
# token is an int from the tokenizer
# idx is our index, to use in our embedding

token2idx = {0: 0}
idx2token = {}

all_tokens = [
    element
    for list_ids in tokenized_datasets["train"]["input_ids"]
    for element in list_ids
]
all_tokens = list(set(all_tokens))

token_index = 0
for token in all_tokens:
    if token not in token2idx:
        token2idx[token] = token_index
        idx2token[token_index] = token
        token_index += 1


def filterSplit(splited_dataset):
    """For valid and test datasets, get only those which all inpu_ids is in splited_dataset['train']"""

    for split in ["validation", "test"]:
        # Filter the splited_dataset[split] to only keep the ids which are in splited_dataset['train']
        splited_dataset[split] = splited_dataset[split].filter(
            lambda x: all(token in token2idx for token in x["input_ids"])
        )

    return splited_dataset


filtered_datasets = filterSplit(tokenized_datasets)
filtered_datasets

Filter:   0%|          | 0/1154 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1154 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 9233
    })
    validation: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 853
    })
    test: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 844
    })
})

Vamos construir nosso dicionário de tokens

In [28]:
def makeIndex(batch):
    batch["input_idx"] = [token2idx[single] for single in batch["input_ids"]]
    return batch

data = filtered_datasets.map(makeIndex, batched=False)
data

Map:   0%|          | 0/9233 [00:00<?, ? examples/s]

Map:   0%|          | 0/853 [00:00<?, ? examples/s]

Map:   0%|          | 0/844 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask', 'input_idx'],
        num_rows: 9233
    })
    validation: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask', 'input_idx'],
        num_rows: 853
    })
    test: Dataset({
        features: ['sentence', 'label', 'input_ids', 'attention_mask', 'input_idx'],
        num_rows: 844
    })
})

In [29]:
# Assert all same length
list(set([len(l_idx) for l_idx in data['train']['input_idx']]))

[80]

In [30]:
data_train = torch.Tensor(data['train']['input_idx']).type(torch.int32)
data_train.shape

torch.Size([9233, 80])

In [31]:
import torch.nn.functional as F
target_train = torch.Tensor(data['train']['label'])

#make one_hot
target_train = F.one_hot(target_train.to(torch.int64), num_classes=N_CLASSES).float()

target_train[:10]

tensor([[0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])

In [32]:
data_valid = torch.Tensor(data['validation']['input_idx']).type(torch.int32)
data_valid.shape

torch.Size([853, 80])

In [33]:
target_valid = torch.Tensor(data['validation']['label'])

#make one_hot
target_valid = F.one_hot(target_valid.to(torch.int64), num_classes=N_CLASSES).float()

target_valid[:10]

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.]])

In [34]:
data_test = torch.Tensor(data['test']['input_idx']).type(torch.int32)
data_test.shape

torch.Size([844, 80])

In [35]:
target_test = torch.Tensor(data['test']['label'])

#make one_hot
target_test = F.one_hot(target_test.to(torch.int64), num_classes=N_CLASSES).float()

target_test[:10]

tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])

## Now let's train our model! 

In [36]:
import torch

log_interval = 10
EPOCHS = 10


def evaluate(model, eval_loader, criterion, verbose=False):
    model = model.to(device)
    model.eval()
    loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, target in eval_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            
            if type(criterion) == torch.nn.modules.loss.NLLLoss:
                loss += criterion(output, target.argmax(dim=-1)).item()
            else:
                loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            target_ = target.argmax(dim=1, keepdim=True)
            correct += pred.eq(target_).sum()

    # loss /= len(eval_loader.dataset)
    verbose and print(
        "\nAverage loss: {:.6f}, Accuracy: {}/{} ({:.2f}%)\n".format(
            loss,
            correct,
            len(eval_loader.dataset),
            correct / len(eval_loader.dataset) * 100,
        )
    )
    return loss


def train(
    model, train_loader, optimizer, criterion, epoch
):
    model = model.to(device)
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        if type(criterion) == torch.nn.modules.loss.NLLLoss:
            loss = criterion(output, target.argmax(dim=-1))
        else:
            loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # valid_loss = evaluate(model, validation_loader, criterion)
        # if valid_loss < best_loss:
        #     best_loss = valid_loss
        #     torch.save(model.state_dict(), "model.pth")

        if (batch_idx + 1) % log_interval == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(data),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )


class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]


model = ClassifierEncoder(vocab_size=len(token2idx))

train_dataset = MyDataset(data_train, target_train)
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True
)

valid_dataset = MyDataset(data_valid, target_valid)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE)

test_dataset = MyDataset(data_train, target_train)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

# TODO: get optimizer and criterion from literature
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.NLLLoss()

best_loss = np.inf
losses = []
if TRAIN_MODEL:
    for epoch in range(1, EPOCHS + 1):
        train(model, train_loader, optimizer, criterion, epoch)
        loss = evaluate(model, valid_loader, criterion, verbose=True)
        losses.append(loss)
        if loss < best_loss:
            best_loss = loss
            torch.save(model.state_dict(), "model.pth")
        print(f"Epoch: {epoch}\tloss: {loss}")
    

# Unit tests for transformer and attention

In [37]:
import unittest

class TestTensorFunctions(unittest.TestCase):
    def test_repeat(self):
        x = torch.tensor([1, 2, 3])
        n = 2
        result = repeat(x, n)
        expected = torch.tensor([[1, 2, 3], [1, 2, 3]])
        self.assertTrue(torch.equal(result, expected))

    def test_batched_matmul(self):
        tensor_3d = torch.randn(10, 3, 4)
        tensor_2d = torch.randn(4, 5)
        result = batched_matmul(tensor_3d, tensor_2d)
        expected = torch.bmm(tensor_3d, tensor_2d.unsqueeze(0).repeat((10, 1, 1)))
        self.assertTrue(torch.allclose(result, expected))

In [38]:
import unittest

class EmbeddingLayerTest(unittest.TestCase):
    def setUp(self):
        # Set up your embedding layer or load your model
        self.vocab_size = 10
        self.d_model = 8
        self.batch_size = 2
        self.t = 4
        self.model = Embedding(self.vocab_size, self.d_model)
        self.criterion = nn.MSELoss()

    def test_embedding_layer_forward(self):
        # Define input tensor (batch_size=2, sequence_length=4)
        x = torch.tensor([[1, 3, 5, 7], [0, 2, 4, 6]])
        
        # Manually set model weights for the embedding matrix
        self.model.embedding.data = torch.tensor([
            [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
            [0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6],
            [1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4],
            [2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2],
            [3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0],
            [4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8],
            [4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6],
            [5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4],
            [6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2],
            [7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0]
        ])
        
        # Calculate the expected output manually
        expected_output = torch.tensor([
            [
                [
                    [0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6],
                    [2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2],
                    [4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8],
                    [5.7, 5.8, 5.9, 6.0, 6.1, 6.2, 6.3, 6.4]
                ],
                [
                    [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
                    [1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4],
                    [3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0],
                    [4.9, 5.0, 5.1, 5.2, 5.3, 5.4, 5.5, 5.6]
                ]
            ]
        ])
        
        # Check if the shape of the expected output matches
        self.assertEqual(expected_output.shape, expected_output.shape)
        
        # Check if the values of the expected output tensor match
        self.assertTrue(torch.allclose(expected_output, expected_output, atol=1e-6))

    def test_embedding_layer_loss(self):
        # Define input tensors and target tensors
        input_tensor = torch.randint(0, self.vocab_size, (self.batch_size, self.t)).long()
        target_tensor = torch.randn(self.batch_size, self.t, self.d_model)

        # Forward pass through the embedding layer
        output = self.model(input_tensor)

        # Compute the loss
        loss = self.criterion(output, target_tensor)

        # Define an optimizer
        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.01)

        # Save the parameters before the backward pass and parameter update
        params_before = [param.clone() for param in self.model.parameters()]

        # Backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Check if the parameters have been updated
        for param, param_before in zip(self.model.parameters(), params_before):
            self.assertTrue(torch.any(param.grad != 0), "No gradients in the parameters")
            self.assertTrue(torch.any(param != param_before), "Parameters have not been updated")
            
    
# This will run the test in the notebook
# RUN_UNIT_TESTS and unittest.main(argv=[""], exit=False)

In [39]:
class TestAttention(unittest.TestCase):
    """_summary_

    Args:
        unittest (_type_): unit test for a conjuntion linear layer
            plus the attention block
    """


class AttentionTest(unittest.TestCase):
    def setUp(self):
        self.batch_size = 2
        self.d_k = 3
        self.d_v = 3
        self.T = 2
        self.d_model = 4

        # Set up your attention mechanism or load your model
        self.model = Attention(self.d_k, self.d_v, self.d_model)
        self.criterion = nn.MSELoss()

    def test_attention_forward(self):
        # Define input tensor (batch_size=2, sequence_length=2, d_model=4)
        x = torch.tensor(
            [
                [[1.0, 2.0, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0]],
                [[9.0, 10.0, 11.0, 12.0], [13.0, 14.0, 15.0, 16.0]],
            ]
        )

        # Manually set model weights for W_K, W_Q, and W_V
        self.model.W_K.data = torch.tensor(
            [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9], [1.0, 1.1, 1.2]],
            dtype=torch.float32,
        )
        self.model.W_Q.data = torch.tensor(
            [[1.2, 1.1, 1.0], [0.9, 0.8, 0.7], [0.6, 0.5, 0.4], [0.3, 0.2, 0.1]],
            dtype=torch.float32,
        )
        self.model.W_V.data = torch.tensor(
            [[0.5, 0.6, 0.7], [0.8, 0.9, 1.0], [1.1, 1.2, 1.3], [1.4, 1.5, 1.6]],
            dtype=torch.float32,
        )

        # Expected output based on calculations from the attention mechanism
        expected_output, _ = self.model(x)

        # Calculate the expected output manually
        # Calculate Q, K, and V using learned weights
        Q = torch.matmul(x, self.model.W_Q)
        K = torch.matmul(x, self.model.W_K)
        V = torch.matmul(x, self.model.W_V)

        # Calculate the attention scores
        attention_scores = torch.matmul(Q, K.transpose(1, 2)) / (self.d_k**0.5)
        attention_weights = torch.nn.functional.softmax(attention_scores, dim=-1)

        # Apply attention weights to V to get the output
        expected_output_manual = torch.matmul(attention_weights, V)

        # Check if the shape of the expected output matches
        self.assertEqual(expected_output.shape, expected_output_manual.shape)

        # Check if the values of the expected output tensor match
        self.assertTrue(
            torch.allclose(expected_output, expected_output_manual, atol=1e-6)
        )

    def test_attention_loss(self):
        # Test the loss function associated with the attention mechanism
        # Define input tensors and target tensors

        input_tensor = torch.rand(
            (self.batch_size, self.T, self.d_model), requires_grad=True
        )

        # Forward pass through the attention mechanism
        output, _ = self.model(input_tensor)

        # Create dummy target tensor (replace with your actual target tensor)
        target = torch.rand_like(output, requires_grad=False)

        # Calculate the loss
        loss = self.criterion(output, target)

        # Define an optimizer
        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.01)

        # Save the parameters before the backward pass and parameter update
        params_before = [param.clone() for param in self.model.parameters()]

        # Perform backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Check if gradients are computed
        self.assertTrue(
            input_tensor.grad is not None, "No gradients in the input tensor"
        )

        # Check if the parameters have been updated
        for param, param_before in zip(self.model.parameters(), params_before):
            self.assertTrue(
                torch.any(param != param_before), "Parameters have not been updated"
            )


# This will run the test in the notebook
# RUN_UNIT_TESTS and unittest.main(argv=[""], exit=False)

In [40]:
# Code by GPT4 to calculate the multihead attention output

import torch

# Define the parameters
d_k = 3
d_v = 3
d_model = 4
h = 2
batch_size = 2
t = 2

# Define the input tensor
input_tensor = torch.tensor([[[1.0, 2.0, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0]],
                             [[9.0, 10.0, 11.0, 12.0], [13.0, 14.0, 15.0, 16.0]]], dtype=torch.float32)

# Define the weights
W_K = torch.tensor([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9], [1.0, 1.1, 1.2]], dtype=torch.float32)
W_Q = torch.tensor([[0.2, 0.3, 0.4], [0.5, 0.6, 0.7], [0.8, 0.9, 1.0], [1.1, 1.2, 1.3]], dtype=torch.float32)
W_V = torch.tensor([[0.3, 0.4, 0.5], [0.6, 0.7, 0.8], [0.9, 1.0, 1.1], [1.2, 1.3, 1.4]], dtype=torch.float32)
W_O = torch.tensor([[0.1, 0.2, 0.3, 0.4], [0.5, 0.6, 0.7, 0.8], 
                    [0.9, 1.0, 1.1, 1.2], [1.3, 1.4, 1.5, 1.6],
                    [1.7, 1.8, 1.9, 2.0], [2.1, 2.2, 2.3, 2.4]], dtype=torch.float32)

# Initialize tensors for K, Q, V for each head
Ks, Qs, Vs = [], [], []

# Compute K, Q, V for each head
for _ in range(h):
    K = torch.matmul(input_tensor, W_K)
    Q = torch.matmul(input_tensor, W_Q)
    V = torch.matmul(input_tensor, W_V)
    Ks.append(K)
    Qs.append(Q)
    Vs.append(V)

# Initialize a tensor for the concatenated results
concatenated_results = torch.zeros(batch_size, t, h * d_v)

# Calculate the attention and concatenate results for each head
for i in range(h):
    # Scaled dot product of Q and K
    attention_scores = torch.matmul(Qs[i], Ks[i].transpose(-2, -1)) / (d_k ** 0.5)
    attention_weights = torch.nn.functional.softmax(attention_scores, dim=-1)
    
    # Weighted sum of V
    head_output = torch.matmul(attention_weights, Vs[i])
    
    # Concatenate results
    concatenated_results[:, :, i * d_v: (i + 1) * d_v] = head_output

# Apply the final linear layer W_O
expected_output = torch.matmul(concatenated_results, W_O)

print(expected_output)

tensor([[[159.9200, 174.0800, 188.2400, 202.4000],
         [159.9200, 174.0800, 188.2400, 202.4000]],

        [[344.5600, 375.0400, 405.5200, 436.0000],
         [344.5600, 375.0400, 405.5200, 436.0000]]])


In [50]:
class MultiHeadAttentionTest(unittest.TestCase):
    def setUp(self):
        self.batch_size = 2
        self.d_k = 3
        self.d_v = 3
        self.t = 2
        self.d_model = 4
        self.h = 2

        # Initialize the multi-head attention model
        self.model = MultiHeadAttention(self.h, self.d_model, self.d_k, self.d_v)
        self.criterion = nn.MSELoss()

        # Manually setting the weights for the test
        for head in self.model.attentions:
            # These weights should be carefully chosen to ensure a predictable output
            head.W_K = nn.Parameter(
                torch.tensor(
                    [[0.1, 0.2, 0.3], [0.4, 0.5, 0.6], [0.7, 0.8, 0.9], [1.0, 1.1, 1.2]]
                )
            )
            head.W_Q = nn.Parameter(
                torch.tensor(
                    [[0.2, 0.3, 0.4], [0.5, 0.6, 0.7], [0.8, 0.9, 1.0], [1.1, 1.2, 1.3]]
                )
            )
            head.W_V = nn.Parameter(
                torch.tensor(
                    [[0.3, 0.4, 0.5], [0.6, 0.7, 0.8], [0.9, 1.0, 1.1], [1.2, 1.3, 1.4]]
                )
            )

        # Output linear layer weights
        self.model.W_O = nn.Parameter(
            torch.tensor(
                [
                    [0.1, 0.2, 0.3, 0.4],
                    [0.5, 0.6, 0.7, 0.8],
                    [0.9, 1.0, 1.1, 1.2],
                    [1.3, 1.4, 1.5, 1.6],
                    [1.7, 1.8, 1.9, 2.0],
                    [2.1, 2.2, 2.3, 2.4],
                ],
                dtype=torch.float32,
            )
        )

        # Shape of W_O, according to the paper "Attention Is All You Need"
        self.assertEqual(list(self.model.W_O.shape), [self.h * self.d_v, self.d_model])

    def test_multi_head_attention_forward(self):
        # Define a specific input tensor (batch_size, t, d_model)
        input_tensor = torch.tensor(
            [
                [[1.0, 2.0, 3.0, 4.0], [5.0, 6.0, 7.0, 8.0]],
                [[9.0, 10.0, 11.0, 12.0], [13.0, 14.0, 15.0, 16.0]],
            ]
        )

        # Expected output tensor based on the input and the model weights
        # This tensor should be calculated manually based on the model's operations
        expected_output = torch.Tensor(
            [
                [
                    [159.9200, 174.0800, 188.2400, 202.4000],
                    [159.9200, 174.0800, 188.2400, 202.4000],
                ],
                [
                    [344.5600, 375.0400, 405.5200, 436.0000],
                    [344.5600, 375.0400, 405.5200, 436.0000],
                ],
            ]
        )

        # Run the forward pass
        output, _ = self.model(input_tensor)

        # Check the shape of the output tensor
        self.assertEqual(output.shape, (self.batch_size, self.t, self.d_model))

        # Check the values of the output tensor
        self.assertTrue(torch.allclose(output, expected_output, atol=1e-4))

    def test_multi_head_attention_loss(self):
        # Test the loss function associated with the multi-head attention mechanism
        # Define input tensors and target tensors

        input_tensor = torch.rand(
            (self.batch_size, self.t, self.d_model), requires_grad=True
        )

        # Forward pass through the multi-head attention mechanism
        output, _ = self.model(input_tensor)

        # Create dummy target tensor (replace with your actual target tensor)
        target = torch.rand_like(output, requires_grad=False)

        # Calculate the loss
        loss = self.criterion(output, target)

        # Define an optimizer
        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.01)

        # Save the parameters before the backward pass and parameter update
        params_before = [param.clone() for param in self.model.parameters()]

        # Perform backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Check if gradients are computed
        self.assertTrue(
            input_tensor.grad is not None, "No gradients in the input tensor"
        )

        # Check if the parameters have been updated
        for param, param_before in zip(self.model.parameters(), params_before):
            self.assertTrue(
                torch.any(param != param_before), "Parameters have not been updated"
            )


# This will run the test in the notebook
# RUN_UNIT_TESTS and unittest.main(argv=[""], exit=False)

........FF
FAIL: test_transformer_block_forward (__main__.TransformerBlockTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_80174/1159015403.py", line 63, in test_transformer_block_forward
    self.assertTrue(torch.allclose(output, expected_output, atol=1e-6))
AssertionError: False is not true

FAIL: test_transformer_block_loss (__main__.TransformerBlockTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_80174/1159015403.py", line 101, in test_transformer_block_loss
    self.assertTrue(
AssertionError: tensor(False) is not true : Parameters have not been updated

----------------------------------------------------------------------
Ran 10 tests in 0.013s

FAILED (failures=2)


In [51]:
# Code by GPT4 to calculate the transformer block output
import torch
import torch.nn.functional as F

# Parameters for the setup
batch_size = 2
d_k = 3
d_v = 3
t = 4
d_model = 3
h = 2

# Define the specific input tensor
x = torch.tensor([[[1.0, 2.0, 3.0],
                   [4.0, 5.0, 6.0],
                   [7.0, 8.0, 9.0],
                   [10.0, 11.0, 12.0]],
                  [[13.0, 14.0, 15.0],
                   [16.0, 17.0, 18.0],
                   [19.0, 20.0, 21.0],
                   [22.0, 23.0, 24.0]]], dtype=torch.float32)

# Predefined weights for the model
W_Ks = [torch.full((d_model, d_k), 0.1 * (i + 1), dtype=torch.float32) for i in range(h)]
W_Qs = [torch.full((d_model, d_k), 0.2 * (i + 1), dtype=torch.float32) for i in range(h)]
W_Vs = [torch.full((d_model, d_v), 0.3 * (i + 1), dtype=torch.float32) for i in range(h)]
W_O = torch.full((h * d_v, d_model), 0.5, dtype=torch.float32)

# Function to calculate the attention output for one head
def attention_output(x, W_K, W_Q, W_V):
    K = torch.matmul(x, W_K)
    Q = torch.matmul(x, W_Q)
    V = torch.matmul(x, W_V)

    scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))
    attn_weights = F.softmax(scores, dim=-1)
    return torch.matmul(attn_weights, V)

# Calculating the output for each head
heads_output = [attention_output(x, W_Ks[i], W_Qs[i], W_Vs[i]) for i in range(h)]

# Concatenating the output of all heads
concatenated_output = torch.cat(heads_output, dim=-1)

# Applying the final linear layer
expected_output = torch.matmul(concatenated_output, W_O)

expected_output

tensor([[[43.8172, 43.8172, 43.8172],
         [44.5119, 44.5119, 44.5119],
         [44.5477, 44.5477, 44.5477],
         [44.5499, 44.5499, 44.5499]],

        [[93.1500, 93.1500, 93.1500],
         [93.1500, 93.1500, 93.1500],
         [93.1500, 93.1500, 93.1500],
         [93.1500, 93.1500, 93.1500]]])

In [55]:
class TransformerBlockTest(unittest.TestCase):
    def setUp(self):
        self.batch_size = 2
        self.d_k = 3
        self.d_v = 3
        self.t = 4
        self.d_model = 3
        self.h = 2

        # Initialize the Transformer block model
        self.model = TransformerBlock(self.t, self.d_k, self.d_v, self.d_model, self.h)

        # Set the model's weights to predefined values
        with torch.no_grad():
            for i in range(self.h):
                self.model.mha.attentions[i].W_K.fill_(0.1 * (i + 1))
                self.model.mha.attentions[i].W_Q.fill_(0.2 * (i + 1))
                self.model.mha.attentions[i].W_V.fill_(0.3 * (i + 1))
            self.model.mha.W_O.fill_(0.5)

        self.criterion = nn.MSELoss()

    def test_transformer_block_forward(self):
        # Define a specific input tensor
        x = torch.tensor(
            [
                [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0], [10.0, 11.0, 12.0]],
                [
                    [13.0, 14.0, 15.0],
                    [16.0, 17.0, 18.0],
                    [19.0, 20.0, 21.0],
                    [22.0, 23.0, 24.0],
                ],
            ]
        )

        # Define the expected output tensor based on the specific input and model weights
        # This tensor should be calculated based on the "Attention is all you need" paper, given the input and weights
        expected_output = torch.Tensor(
            [
                [
                    [43.8172, 43.8172, 43.8172],
                    [44.5119, 44.5119, 44.5119],
                    [44.5477, 44.5477, 44.5477],
                    [44.5499, 44.5499, 44.5499],
                ],
                [
                    [93.1500, 93.1500, 93.1500],
                    [93.1500, 93.1500, 93.1500],
                    [93.1500, 93.1500, 93.1500],
                    [93.1500, 93.1500, 93.1500],
                ],
            ]
        )

        # Forward pass
        output = self.model(x)

        # Check if the output matches the expected shape
        self.assertEqual(output.shape, expected_output.shape)

        # Check if the values of the output tensor match the expected values
        self.assertTrue(torch.allclose(output, expected_output, atol=1e-6))


    def test_transformer_block_loss(self):
        # Test the loss function associated with the Transformer block
        # Define input tensors and target tensors

        input_tensor = torch.rand(
            (self.batch_size, self.t, self.d_model), requires_grad=True
        )

        # Forward pass through the Transformer block
        output = self.model(input_tensor)

        # Create dummy target tensor (replace with your actual target tensor)
        target = torch.rand_like(output, requires_grad=False)

        # Calculate the loss
        loss = self.criterion(output, target)

        # Define an optimizer
        optimizer = torch.optim.SGD(self.model.parameters(), lr=0.01)

        # Save the parameters before the backward pass and parameter update
        params_before = [param.clone() for param in self.model.parameters()]

        # Perform backward pass
        loss.backward()

        # Update the parameters
        optimizer.step()

        # Check if gradients are computed
        self.assertTrue(
            input_tensor.grad is not None, "No gradients in the input tensor"
        )

        # Check if the parameters have been updated
        for param, param_before in zip(self.model.parameters(), params_before):
            self.assertTrue(
                torch.any(param != param_before), "Parameters have not been updated"
            )


# This will run the test in the notebook
RUN_UNIT_TESTS and unittest.main(argv=[""], exit=False)

........FF
FAIL: test_transformer_block_forward (__main__.TransformerBlockTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_80174/1310023453.py", line 63, in test_transformer_block_forward
    self.assertTrue(torch.allclose(output, expected_output, atol=1e-6))
AssertionError: False is not true

FAIL: test_transformer_block_loss (__main__.TransformerBlockTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_80174/1310023453.py", line 102, in test_transformer_block_loss
    self.assertTrue(
AssertionError: tensor(False) is not true : Parameters have not been updated

----------------------------------------------------------------------
Ran 10 tests in 0.014s

FAILED (failures=2)


In [47]:

# This will run the test in the notebook
RUN_UNIT_TESTS and unittest.main(argv=[""], exit=False)

....FF..FF
FAIL: test_multi_head_attention_forward (__main__.MultiHeadAttentionTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_80174/1397261406.py", line 49, in setUp
    self.assertEqual(list(self.model.W_O.shape), [h * d_v, d_model])
AssertionError: Lists differ: [6, 4] != [6, 3]

First differing element 1:
4
3

- [6, 4]
?     ^

+ [6, 3]
?     ^


FAIL: test_multi_head_attention_loss (__main__.MultiHeadAttentionTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipykernel_80174/1397261406.py", line 49, in setUp
    self.assertEqual(list(self.model.W_O.shape), [h * d_v, d_model])
AssertionError: Lists differ: [6, 4] != [6, 3]

First differing element 1:
4
3

- [6, 4]
?     ^

+ [6, 3]
?     ^


FAIL: test_transformer_block_forward (__main__.TransformerBlockTest)
-----------------------------------------------------------------